## Data

In [1]:
import numpy as np
import pandas as pd
from pandasql import sqldf
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

In [2]:
file_name = 'p-2021-11-13 raw log.csv'
df = pd.read_csv(f'./data/processed/{file_name}')

In [3]:
df['hand_num'] = pd.to_numeric(df['hand_num'])
df['at'] = pd.to_datetime(df['at'])

## Analysis

### Games statistics

1. What are the pot size distributions?

In [11]:
df.groupby('hand_num')['pot_size'].max().sort_values(ascending=False).to_frame().hist(title='Distribution of pot size')

Unsurprisingly the distribution is heavily skewed. The majority of pots are pretty small, in the $0-$20 range. There are only 5 hands where the pot exceeded $100.

Perhaps it's easier to group the pot sizes by bins and count:

In [15]:
pd.cut(df.groupby('hand_num')['pot_size'].max(), bins=[0, 5, 10, 20, 50, 100, np.inf]).value_counts().sort_index()

(0.0, 5.0]       131
(5.0, 10.0]       92
(10.0, 20.0]      54
(20.0, 50.0]      27
(50.0, 100.0]      2
(100.0, inf]       3
Name: pot_size, dtype: int64

2. How many pots were split?

In [16]:
df.loc[df['action']=='Won']['hand_num'].value_counts().value_counts()

1    302
2      6
Name: hand_num, dtype: int64

3. At which phases are the hands won?

In [17]:
df.loc[df['action']=='Won'].groupby('hand_num')['phase'].max().value_counts(dropna=False)

River       183
Turn         55
Flop         52
Pre-flop     18
Name: phase, dtype: int64

Judging from the result, we can conclude that the game was pretty casual as it often went to River. We can contrast this distribution vs. a higher stake game to further illustrate the point.

### Individual player statistics

0. Number of hands played

In [52]:
# pl = player dataframe to store all metrics
pl = df.loc[(df['phase']=='Pre-flop') & (df['rank-P1']==1)]['player_name'].value_counts(dropna=False).to_frame(name='num_hands_played')
pl

,num_hands_played
HATuan @ Yf-9wDC92J,484
HoangLanHN @ Kgof11uGzY,483
Loi @ 6KDaksPWKW,481
Dory @ NaZSSnTRkj,459
Honda @ 59Y3kR2GJJ,400
Lelouch @ wNYgotQI9c,358
sonximang @ J9dq4AB4dI,172


1. Sum of pot wins & net PnL

In [26]:
df.loc[df['action']=='Won'].groupby('player_name')['_amount'].sum().sort_values().plot(kind='barh', title='Gross winnings')

We can make a reasonable assumption that, given enough hands, the better players win more. Next we will analyse different patterns of behaviors. Hopefully we can learn from the winners here.

1. VPIP: Voluntarily Put In Pot

VPIP tracks the percentage of hands in which a particular player voluntarily puts money into the pot preflop. VPIP increases when a player could fold but instead commits money to the pot preflop. This includes limping (merely calling the big blind), calling, and raising.



In [53]:
_vpip = df.loc[(df['phase']=='Pre-flop') &
    (df['rank-P2']==1) &
    (df['action'].isin(['Bets/Raises', 'Calls']))
    ]['player_name'].value_counts(dropna=False)
_vpip.name = 'VPIP'
pl = pl.merge(_vpip, how='left', left_index=True, right_index=True)
pl['% VPIP'] = (pl['VPIP'] / pl['num_hands_played']).round(2)


In [46]:
pl.sort_values(by='% VPIP')['% VPIP'].plot(kind='barh', title='VPIP')

2. PFR: Pre-flop Raises

PFR tracks the percentage of hands in which a particular player makes a preflop raise when having the opportunity to fold or call instead. This includes reraises. By definition, PFR is a subset of VPIP. So we will plot the ratio PFR/VPIP to measure player's tendency.

In [59]:
_pfr = df.loc[(df['phase']=='Pre-flop') &
    (df['action']=='Bets/Raises')][['player_name', 'hand_num']].drop_duplicates()['player_name'].value_counts()
_pfr.name = 'PFR'
pl = pl.merge(_pfr, how='left', left_index=True, right_index=True)

In [66]:
(pl['PFR'] / pl['VPIP']).round(3).sort_values().plot(kind='barh', title='PFR/VPIP')

3. 3-BET

In [54]:
_3bet = df.loc[(df['phase']=='Pre-flop') &
    (df['action']=='Bets/Raises') &
    (df['rank-A2']>1)]['player_name'].value_counts(dropna=False).fillna(0)
_3bet.name = '3-BET'
pl = pl.merge(_3bet, how='left', left_index=True, right_index=True)
pl['% 3-BET'] = (pl['3-BET'] / pl['num_hands_played']).round(3)

In [67]:
pl.sort_values(by='% 3-BET', na_position='first')['% 3-BET'].plot(kind='barh', title='3-BET')

In [51]:
pl

,num_hands_played,VPIP,% VPIP,3-BET,% 3-BET
HATuan @ Yf-9wDC92J,484,169,0.35,1.0,0.00
HoangLanHN @ Kgof11uGzY,483,178,0.37,9.0,0.02
Loi @ 6KDaksPWKW,481,146,0.30,7.0,0.01
Dory @ NaZSSnTRkj,459,177,0.39,2.0,0.00
Honda @ 59Y3kR2GJJ,400,203,0.51,3.0,0.01
Lelouch @ wNYgotQI9c,358,96,0.27,4.0,0.01
sonximang @ J9dq4AB4dI,172,67,0.39,NaN,NaN


4. C-BET

5. Wins based on positions

Good players play positions as well!

In [58]:
df.loc[df['action']=='Won'].pivot_table(
    index='player_name',
    columns='position_tag',
    values='hand_num',
    aggfunc=len,
    fill_value=0,
    margins=True
)

position_tag,Dealer,EP,LP,MP,All
player_name,,,,,
Dory @ NaZSSnTRkj,6,8,20,15,49
HATuan @ Yf-9wDC92J,10,7,17,17,51
HoangLanHN @ Kgof11uGzY,3,17,16,15,51
Honda @ 59Y3kR2GJJ,9,7,12,18,46
Lelouch @ wNYgotQI9c,3,4,11,12,30
Loi @ 6KDaksPWKW,19,14,17,23,73
sonximang @ J9dq4AB4dI,3,6,5,0,14
All,53,63,98,100,314
